In [4]:
import random
from top2vec_modified.Top2Vec import Top2Vec, default_tokenizer
import yaml

def generate_docs(length=50, rep=20):
    vocabs = [
        ["lorem", "ipsum", "dolor"],
        ["lorem", "sit", "amet"],
        ["lorem", "sit", "adipiscing", "eiusmod"],
        ["lorem", "consectetur", "adipiscing"],
        ["lorem", "consectetur", "elit"],
        ["lorem", "consectetur", "eiusmod"],
        ["lorem", "consectetur", "eiusmod"],
    ]

    return [" ".join(random.choice(vocab) for _ in range(length)) for _ in range(rep) for vocab in vocabs]

In [4]:

#with open("top2vec_modified/Top2Vec_config.yaml") as f:
#    config = yaml.safe_load(f)

#docs = generate_docs()

#config.update({"use_pacmap": True, "dimred_args": {}})
#config.update({"embedding_model": "universal-sentence-encoder"})

#model = Top2Vec(documents=docs, **config, plain=False)

In [44]:
docs = generate_docs(rep=80, length=100)
a = Top2Vec.run(documents=docs, config_modifier={"phases": ["create_embeddings"]})
a.save("test_model.model")

2024-03-07 08:28:26,932 - top2vec - INFO - Pre-processing documents for training
2024-03-07 08:28:27,026 - top2vec - INFO - Creating joint document/word embedding


In [45]:
b = Top2Vec.run(from_model_path='test_model.model', config_modifier={"phases": ["reduce_dimension", "discover_topics"]})

2024-03-07 08:28:28,683 - top2vec - INFO - Creating lower dimension embedding of documents
2024-03-07 08:28:30,565 - top2vec - INFO - Finding dense areas of documents
2024-03-07 08:28:30,573 - top2vec - INFO - Finding topics


In [46]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

doc2vec_args = {"vector_size": 300,
                "min_count": 5,
                "window": 15,
                "sample": 1e-5,
                "negative": 0,
                "hs": 1,
                "epochs": 40,
                "dm": 0,
                "dbow_words": 1}

train_corpus = [TaggedDocument(default_tokenizer(doc), [i]) for i, doc in enumerate(docs)]
doc2vec_args["documents"] = train_corpus

doc2vec = Doc2Vec(**doc2vec_args)
doc2vec.save("doc2vec_test.model")

In [47]:
doc2vec = Doc2Vec.load("doc2vec_test.model")
top2vec = Top2Vec.run(documents=docs, config_modifier={"embedding_model": doc2vec.infer_vector, "phases": ["create_embeddings"]})

2024-03-07 08:28:32,805 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2024-03-07 08:28:32,894 - top2vec - INFO - Creating joint document/word embedding


In [52]:
b.word_vectors.shape

(9, 300)

In [57]:
top2vec.document_vectors.shape

(18, 300)

In [53]:
top2vec_use = Top2Vec.run(documents=docs, config_modifier={"embedding_model": "universal-sentence-encoder", "phases": ["create_embeddings"]})


2024-03-07 08:32:57,644 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2024-03-07 08:32:57,722 - top2vec - INFO - Downloading universal-sentence-encoder model
2024-03-07 08:32:57.773778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-07 08:32:57.777095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/

In [56]:
top2vec_use.document_vectors.shape

(560, 512)